In [28]:
import pandas as pd
import numpy as np
import requests
import json

from pandas.io.json import json_normalize

In [2]:
toronto_area = pd.read_excel('data/Toronto_Wards_Area.xlsx')

In [3]:
toronto_area_clean = toronto_area.rename(columns={'City of Toronto Profiles':'Ward_ID', 'Unnamed: 1':'Area'})
toronto_area_clean.drop(list(np.arange(0,11)), inplace = True)
toronto_area_clean.reset_index(drop = True, inplace = True)
toronto_area_clean.dtypes

Ward_ID    object
Area       object
dtype: object

In [4]:
toronto_area_clean.astype({'Ward_ID':'int64', 'Area':'float64'}).dtypes
print("The shape of 'toronto_area_clean': ", toronto_area_clean.shape)
toronto_area_clean.head()

The shape of 'toronto_area_clean':  (25, 2)


,Ward_ID,Area
0,1,48.38
1,2,37.36
2,3,40.03
3,4,15.33
4,5,24.98


In [5]:
toronto_loc = pd.read_csv('data/Toronto_Wards_Location.csv')

In [6]:
toronto_loc.columns

Index(['_id', 'AREA_ID', 'DATE_EFFECTIVE', 'DATE_EXPIRY', 'AREA_ATTR_ID',
       'AREA_TYPE_ID', 'PARENT_AREA_ID', 'AREA_TYPE', 'AREA_CLASS_ID',
       'AREA_CLASS', 'AREA_SHORT_CODE', 'AREA_LONG_CODE', 'AREA_NAME',
       'AREA_DESC', 'FEATURE_CODE', 'FEATURE_CODE_DESC', 'TRANS_ID_CREATE',
       'TRANS_ID_EXPIRE', 'X', 'Y', 'LONGITUDE', 'LATITUDE', 'OBJECTID',
       'Shape__Area', 'Shape__Length', 'geometry'],
      dtype='object')

In [7]:
loc_columns = ['AREA_SHORT_CODE', 'AREA_NAME', 'LONGITUDE','LATITUDE']
toronto_loc_clean = toronto_loc[loc_columns]
toronto_loc_clean = toronto_loc_clean.rename(columns = {'AREA_SHORT_CODE':'Ward_ID','AREA_NAME':'Ward','LONGITUDE':'Long','LATITUDE':'Lat'})
toronto_loc_clean.dtypes

Ward_ID      int64
Ward        object
Long       float64
Lat        float64
dtype: object

In [8]:
print("The shape of 'toronto_loc_clean': ",toronto_loc_clean.shape)
toronto_loc_clean.head()

The shape of 'toronto_loc_clean':  (25, 4)


,Ward_ID,Ward,Long,Lat
0,7,Humber River-Black Creek,-79.530272,43.750724
1,6,York Centre,-79.467810,43.752565
2,18,Willowdale,-79.416500,43.776886
3,11,University-Rosedale,-79.394317,43.671139
4,19,Beaches-East York,-79.306673,43.689268


In [9]:
toronto_demo = pd.read_excel('data/Toronto_Wards_Demographics_2016.xlsx')

In [10]:
toronto_demo_clean = toronto_demo.drop(list(np.arange(15)))
toronto_demo_clean.set_index('City of Toronto Profiles', inplace = True)
toronto_demo_clean.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
City of Toronto Profiles,,,,,,,,,,,,,,,,,,,,,
Population,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,Toronto,Ward 1,Ward 2,Ward 3,Ward 4,Ward 5,Ward 6,Ward 7,Ward 8,Ward 9,...,Ward 16,Ward 17,Ward 18,Ward 19,Ward 20,Ward 21,Ward 22,Ward 23,Ward 24,Ward 25
Total - Age,2731570,118040,118020,129080,108805,116685,104320,108035,114395,108470,...,94580,110080,118800,109465,110280,112605,105540,98800,102390,102275
0 to 4 years,136000,6815,5230,6370,5770,6530,5510,6575,6300,5020,...,5015,4715,5065,6895,6160,5905,4880,4950,5860,4890
5 to 9 years,135025,7500,5905,5630,5540,6695,5195,6895,7025,4460,...,5185,5140,4420,6550,6330,6250,4875,5125,6385,5565


In [11]:
demo_col_int = list(np.arange(26))
toronto_demo_clean.columns = demo_col_int
toronto_demo_clean.index.name = None
toronto_demo_clean = toronto_demo_clean.T

demo_columns = ['Total - Private households by number of household maintainers - 25% sample data', '  Median total income of households in 2015 ($)']
toronto_demo_clean = toronto_demo_clean[demo_columns]
toronto_demo_clean.reset_index(inplace = True)
toronto_demo_clean.columns = ['Ward_ID', 'Number of Households', 'Income of Households']
# remove the row that contains the sum
toronto_demo_clean.drop([0], inplace = True)
toronto_demo_clean.reset_index(drop = True, inplace = True)
# check column type
toronto_demo_clean.dtypes

Ward_ID                  int64
Number of Households    object
Income of Households    object
dtype: object

In [12]:
toronto_demo_clean.astype('float64').dtypes
print("The shape of 'toronto_demo_clean': ",toronto_demo_clean.shape)
toronto_demo_clean.head()

The shape of 'toronto_demo_clean':  (25, 3)


,Ward_ID,Number of Households,Income of Households
0,1,37895,61169
1,2,45045,82870
2,3,59735,71859
3,4,50315,65546
4,5,44930,53469


In [13]:
from functools import reduce

toronto_dfs = [toronto_area_clean, toronto_loc_clean, toronto_demo_clean]
toronto_df = reduce(lambda left, right: pd.merge(left, right, on = 'Ward_ID', how = 'outer'), toronto_dfs)

toronto_df.set_index('Ward', inplace = True)
toronto_df.drop(columns = ['Ward_ID'], inplace = True)
toronto_df.head()

,Area,Long,Lat,Number of Households,Income of Households
Ward,,,,,
Etobicoke North,48.38,-79.584667,43.719405,37895,61169
Etobicoke Centre,37.36,-79.552534,43.664431,45045,82870
Etobicoke-Lakeshore,40.03,-79.520874,43.621646,59735,71859
Parkdale-High Park,15.33,-79.467340,43.650121,50315,65546
York South-Weston,24.98,-79.493371,43.694951,44930,53469


In [27]:
boston_area = pd.read_csv('data/Boston_Neighborhoods_Area.csv')

area_columns = ['Name', 'Neighborhood_ID', 'SqMiles']
boston_area_clean = boston_area[area_columns]
boston_area_clean.head()

SqMiles_to_SqKm = 2.59
boston_area_clean.loc[:,'SqMiles'] = boston_area_clean.loc[:,'SqMiles'] * SqMiles_to_SqKm
boston_area_clean = boston_area_clean.rename(columns = {'Name':'Neighborhood', 'SqMiles':'Area'})

boston_area_clean.astype({'Neighborhood_ID':'int64', 'Area':'float64'}).dtypes

print("The shape of 'boston_area_clean': ",boston_area_clean.shape)
boston_area_clean.head()

The shape of 'boston_area_clean':  (26, 3)


,Neighborhood,Neighborhood_ID,Area
0,Roslindale,15,6.5009
1,Jamaica Plain,11,10.2046
2,Mission Hill,13,1.4245
3,Longwood,28,0.7511
4,Bay Village,33,0.1036


In [18]:
boston_demo = pd.read_excel('data/Boston_Neighborhood_Demographics_2017.xlsx', sheet_name= 'HH Income', skiprows= 2)

In [25]:
boston_demo_clean = boston_demo.drop([0, 1])
demo_columns = ['Unnamed: 0', 'Median Income', 'Total Households']
boston_demo_clean = boston_demo_clean[demo_columns]
boston_demo_clean.columns = ['Neighborhood', 'Income of Households', 'Number of Households']

boston_demo_clean.reset_index(drop = True, inplace = True)
print("The shape of 'boston_demo_clean': ",boston_demo_clean.shape)
boston_demo_clean.head()

The shape of 'boston_demo_clean':  (27, 3)


,Neighborhood,Income of Households,Number of Households
0,Boston,62021.00,263229.0
1,Allston,46982.76,6457.0
2,Back Bay,102070.55,9824.0
3,Beacon Hill,98069.24,5458.0
4,Brighton,62041.20,21605.0


In [31]:
with open('data/Boston_Neighborhoods_Location.json') as json_data:
    boston_loc_data  = json.load(json_data)

In [47]:
loc_columns = ['Neighborhood_ID', 'Long', 'Lat']
boston_loc = pd.DataFrame(columns= loc_columns)

for loc_data in boston_loc_data:
    neighborhood_id = loc_data['fields']['neighborho']
    longitude = loc_data['fields']['geo_point'][1]
    latitude = loc_data['fields']['geo_point'][0]
    
    boston_loc = boston_loc.append({'Neighborhood_ID':neighborhood_id,
                                    'Long':longitude,
                                    'Lat': latitude}, ignore_index = True)

In [51]:
boston_loc_clean = boston_loc.astype({'Neighborhood_ID':'int64'})
boston_loc_clean.head()

,Neighborhood_ID,Long,Lat
0,13,-71.102606,42.331733
1,6,-71.063456,42.298454
2,29,-71.041726,42.346408
3,12,-71.091446,42.276996
4,28,-71.105102,42.338600
